In [11]:
import time
import os
import glob
import json
import cv2
import math
import numpy as np
from PIL import Image
from IPython.display import clear_output
from statistics import mode
from statistics import mean
from keras.models import load_model

In [12]:
# set parameters
checkpoint_path = r'E:/USGS/checkpoints/final'

# process path
process_path = r'E:/USGS/test/'

# set the confidence threshold for the network
threshold = 0.85

# set the maximum sample factor to attempt per scan
max_sample_factor = 10

In [13]:
# load best model
model = load_model(checkpoint_path)

In [14]:
# define a function to convert 8bit arrays to 1bit arrays that the network can understand
def eight_to_one(crop):
    
    # scale each array to be between 0 and 1
    array = crop/255
    
    # reshape it, network expects <samples, x_dimension, y_dimension, bands>
    array = np.expand_dims(array, axis=2)
    array = np.expand_dims(array, axis=0)

    return array

In [26]:
# use a slightly different function to do the tiling, note the log_tiles flag
def tile_scan(source_path, desired_samples_per_image, desired_tile_dimension, log_tiles):

    # get start time
    start_time = time.time()
    
    # make a dict to store overall results
    results = {}

    # get a list of scans to process
    images_to_process = glob.glob(process_path + '*.TIFF')

    # loop through all the images
    for image in images_to_process:

        # get the image name (adjust if filetype changes, PLEASE READ THIS TIM)
        image_name = image[len(source_path):-5]
        
        # set the tile save path
        tile_path = os.path.join(process_path, image_name)
        
        # print name
        print('Processing:', image_name)
        
        # check if tiles are to be saved
        if log_tiles == True:
            
            # don't create a new directory if it exists
            if not os.path.exists(os.path.join(process_path, image_name)):
                
                # create a subfolder to save tiles
                os.mkdir(tile_path)

        # open the file
        img = Image.open(image)

        # get the dimensions of the image
        imageWidth, imageHeight = img.size
        
        # get the number of pixels per tile
        pixel_count = desired_tile_dimension*desired_tile_dimension

        # sets the x and y ranges 
        rangex = math.ceil(imageWidth / desired_tile_dimension)
        rangey = math.ceil(imageHeight / desired_tile_dimension)
        total_frames = rangex * rangey
        
        # make a counter to count successful samples
        number_of_samples = 0
        
        # make a counter for attempts, used to escape loop if not enough tiles can be made
        samples_attempted = 0 
        
        # make a list to store scan results
        sample_list = []
        
        # set a hard limit on the number of attempts, sometimes images just don't have enough 
        # samples. This limit should be increased if not enough samples are generated.
        while number_of_samples < desired_samples_per_image and samples_attempted < max_sample_factor*desired_samples_per_image: 

            # get random x and y coordinates to sample, stay away from the edges of the scan
            rand_x = np.random.randint(2, rangex-2)
            rand_y = np.random.randint(6, rangey-3)

            # set the crop coordinates. box = (<start x>, <start y>, <end x>, <end y>)
            box = (rand_x*desired_tile_dimension, 
                   rand_y*desired_tile_dimension, 
                   rand_x*desired_tile_dimension+desired_tile_dimension, 
                   rand_y*desired_tile_dimension+desired_tile_dimension)

            # crop each tile
            tile = img.crop(box)

            # get the image as an array
            tile_array = np.asarray(tile)

            # convert to binary
            ret, binary_tile8 = cv2.threshold(tile_array, 127, 255, cv2.THRESH_BINARY)

            # format the crop for network processing
            binary_tile = eight_to_one(binary_tile8)
            
            # ignore mostly white images. We want between 50% and 95% black
            if np.sum(binary_tile) < 0.95*pixel_count and np.sum(binary_tile) > 0.5*pixel_count:

                # note attempted sample
                samples_attempted += 1
                
                # get the classification, the network expects a list
                prediction = model.predict(binary_tile)

                # check that the tile exceeds our confidence threshold
                if np.max(prediction) > threshold:
                    
                    # note successful sample
                    number_of_samples += 1

                    # convert the one-hot encoding to the actual class labels
                    label = np.where(prediction == np.max(prediction))[1][0]

                    # save the prediction and confidence to the results list
                    results_dict = {'x': int(rand_x),
                                    'y': int(rand_y), 
                                    'class_label': int(label),
                                    'confidence': float(np.max(prediction))}
                    
                    # add the sample data to the sample list
                    sample_list.append(results_dict)
                    
                    # check if tiles are to be saved
                    if log_tiles == True:
                        
                        # set the tile save name
                        padding_x_zeros = len(str(rangex))
                        padding_y_zeros = len(str(rangey))
                        filename = '{}_{}_{}-{}.png'.format(image_name, 
                                                            str(rand_y).zfill(padding_y_zeros), 
                                                            str(rand_x).zfill(padding_x_zeros),
                                                            str(label))
                        # save the tile
                        Image.fromarray(binary_tile8).save(os.path.join(tile_path, filename))

            # ignore other images 
            else:

                # note another attempted sample
                samples_attempted += 1
           
            # add to the overall data dict
            results[image_name] = sample_list

        # clear the output 
        clear_output(wait=True)
        
    # save  to a log
    with open(process_path + 'tile_classification_results.json', 'a') as outfile:
        outfile.write(json.dumps(results)) 
        
    # get end time
    end_time = time.time()
    
    print('Processing time: {:.3}s'.format(end_time - start_time))

In [85]:
tile_scan(process_path, 200, 200, log_tiles=False)

Processing time: 5.98e+02s


Took 455s for 74 scans each with 100 samples. ~6s per scan

# Classify each scan

In [86]:
# data path
process_path = r'E:/USGS/test/'

# json log path
json_path = process_path + 'tile_classification_results.json'

In [87]:
# load json
with open(json_path, 'r') as data:
     json_data = json.loads(data.read())

In [88]:
json_data

{'AAE_62_05_22_1522_LHZ': [{'x': 17,
   'y': 10,
   'class_label': 2,
   'confidence': 0.8896721005439758},
  {'x': 16, 'y': 10, 'class_label': 2, 'confidence': 0.9335792064666748},
  {'x': 29, 'y': 6, 'class_label': 0, 'confidence': 0.9312875270843506},
  {'x': 9, 'y': 6, 'class_label': 0, 'confidence': 0.9451307058334351},
  {'x': 21, 'y': 10, 'class_label': 2, 'confidence': 0.8733651638031006},
  {'x': 18, 'y': 8, 'class_label': 2, 'confidence': 0.9421084523200989},
  {'x': 34, 'y': 6, 'class_label': 0, 'confidence': 0.9939684867858887},
  {'x': 35, 'y': 6, 'class_label': 0, 'confidence': 0.9372360706329346},
  {'x': 32, 'y': 6, 'class_label': 0, 'confidence': 0.9583563208580017},
  {'x': 4, 'y': 6, 'class_label': 0, 'confidence': 0.8872097134590149},
  {'x': 20, 'y': 7, 'class_label': 0, 'confidence': 0.9199933409690857},
  {'x': 16, 'y': 6, 'class_label': 0, 'confidence': 0.9478837251663208},
  {'x': 33, 'y': 11, 'class_label': 2, 'confidence': 0.9072251319885254},
  {'x': 6, 'y':

In [89]:
# iterate through the image dictionary
for key in json_data.keys():
    
    # set up a list to store class labels for each scan
    class_label_list = []
    
    # get the image name
    name = key
    
    # get the predictions
    prediction_list = json_data[key]
    
    # loop through the predictions
    for predict_index in range(0, len(prediction_list)):
        
        # get the class
        classification = prediction_list[predict_index]['class_label']
        
        # add it to the list
        class_label_list.append(classification)
    
    # check for guesses
    if len(class_label_list) != 0:
    
        # get some basic statistics 
        scan_mean = mean(class_label_list)
        scan_mode = mode(class_label_list)
        scan_max = max(class_label_list)

        # apply logic to get an overall label for each scan, this can and should be modified
        if scan_max == 0:
            label = 'no interest'

        elif scan_max == 1:
            label = 'little interest'

        elif scan_mode == 2:
            label = 'high interest'
            
        elif scan_max == 2 or scan_mean > 1:
            label = 'interest'
            
    # add an error label
    else:
        label = 'ERROR: could not determine'
        
    # save the results to a log
    with open(process_path + 'scan_classification_results.csv', 'a') as outfile:
        outfile.write('{}, {}, {}, {}, {}\n'.format(name, scan_mean, scan_mode, scan_max, label)) 